# Time Series Kneightbors Regressor Algorithm

This algorithm is used to forecast FOREX prices for the USDCAD pair. It fetches data from MetaTrader5 and applies feature engineering to create additional data.

The forecasted values are set to:
- **Position 1** if the prediction indicates that the price will go up in the next hour
- **Position 0** if the prediction indicates that the price will go down in the nex hourx hour

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-darkgrid")
from matplotlib import cycler
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import MetaTrader5 as mt5
from datetime import datetime
from UsefulFunctions import data, backtest
import ta

In [8]:
# Initiate bound between MetaTrader5 and Python
mt5.initialize()

True

In [10]:
# Fetching data from MetaTrader5 from 2015 to the present using the custom function "get_rates" from the data.py file
# The historical data retrieved is of 1-hour timeframe
df = data.get_rates("USDCAD", mt5.TIMEFRAME_H1, datetime(2015,1,1))
df.head()

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2015-01-02 08:00:00,1.1648,1.1655,1.1646,1.1647,309,40,0
2015-01-02 09:00:00,1.1648,1.1658,1.1647,1.1648,441,40,0
2015-01-02 10:00:00,1.1647,1.1653,1.1642,1.1647,631,40,0
2015-01-02 11:00:00,1.1648,1.1659,1.1642,1.1659,443,40,0
2015-01-02 12:00:00,1.1658,1.1666,1.1655,1.1655,341,40,0


In [12]:
# Drop "spread" and "real_volume" columns and remane columns "tick_volumne" to only "volumne"
df = df[["open", "high", "low", "close", "tick_volume"]]
df.rename(columns={'tick_volume': 'volume'}, inplace=True)
df.head()

,open,high,low,close,volume
time,,,,,
2015-01-02 08:00:00,1.1648,1.1655,1.1646,1.1647,309
2015-01-02 09:00:00,1.1648,1.1658,1.1647,1.1648,441
2015-01-02 10:00:00,1.1647,1.1653,1.1642,1.1647,631
2015-01-02 11:00:00,1.1648,1.1659,1.1642,1.1659,443
2015-01-02 12:00:00,1.1658,1.1666,1.1655,1.1655,341


## Feature Engineering 
**Creating new input variables from raw data**

In [16]:
# Create a second DataFrame from the close column and compute the percent change in a new column
df_copy = df[["close"]]
df_copy["pct_change"] = df_copy["close"].pct_change(1)
df_copy.head()

,close,pct_change
time,,
2015-01-02 08:00:00,1.1647,NaN
2015-01-02 09:00:00,1.1648,0.000086
2015-01-02 10:00:00,1.1647,-0.000086
2015-01-02 11:00:00,1.1659,0.001030
2015-01-02 12:00:00,1.1655,-0.000343


In [36]:
# New DataFrame with all indications from "ta" library and shifting them by one row
df_indicators = ta.add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume", fillna=True).shift(1)
df_indicators

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02 09:00:00,1.16480,1.16550,1.16460,1.16470,309.0,-240.333333,309.0,-0.777778,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.164700,0.000000,0.000000,0.000000
2015-01-02 10:00:00,1.16480,1.16580,1.16470,1.16480,441.0,-601.151515,750.0,-0.801535,0.044100,0.049887,...,0.000685,0.000137,0.000548,3.303215,0.660643,2.642572,1.164745,0.008586,0.008586,0.008586
2015-01-02 11:00:00,1.16470,1.16530,1.16420,1.16470,631.0,-658.515152,119.0,-0.476839,0.028786,-0.087163,...,0.000529,0.000215,0.000313,9.890616,2.506637,7.383978,1.164725,-0.008585,-0.008586,0.000000
2015-01-02 12:00:00,1.16480,1.16590,1.16420,1.16590,443.0,-215.515152,562.0,-0.118155,0.100616,0.115124,...,0.008619,0.001896,0.006723,10.497761,4.104862,6.392898,1.165237,0.103031,0.102978,0.103031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-20 23:00:00,1.36212,1.36227,1.36185,1.36218,1844.0,-441219.257908,2271288.0,-0.100588,0.040593,-0.010136,...,-0.049974,-0.064970,0.014996,16.000455,17.178093,-1.177638,1.362917,0.004405,0.004405,16.955439
2024-08-21 00:00:00,1.36217,1.36221,1.36159,1.36183,540.0,-441341.193392,2270748.0,-0.125629,0.007794,-0.018370,...,-0.050446,-0.062065,0.011620,8.128629,15.368200,-7.239571,1.362872,-0.025694,-0.025697,16.925389
2024-08-21 01:00:00,1.36180,1.36232,1.36160,1.36221,931.0,-440694.665614,2271679.0,-0.095080,0.057220,0.004640,...,-0.048016,-0.059256,0.011239,2.885485,12.871657,-9.986172,1.362847,0.027904,0.027900,16.958015


In [34]:
# Identifying columns with all zero values to avoid interference during model training
zero_columns_list = df_indicators.columns[(df_indicators == 0).all(axis=0)].tolist()
zero_columns_list

[]

In [40]:
# Concatenating both dataframes and drop the close column (which wasn't shifted) from df_copy
new_df = pd.concat((df_indicators, df_copy["pct_change"]), axis=1)
new_df.dropna(inplace=True)
new_df.head()

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,pct_change
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 09:00:00,1.1648,1.1655,1.1646,1.1647,309.0,-240.333333,309.0,-0.777778,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.164700,0.000000,0.000000,0.000000,0.000086
2015-01-02 10:00:00,1.1648,1.1658,1.1647,1.1648,441.0,-601.151515,750.0,-0.801535,0.044100,0.049887,...,0.000137,0.000548,3.303215,0.660643,2.642572,1.164745,0.008586,0.008586,0.008586,-0.000086
2015-01-02 11:00:00,1.1647,1.1653,1.1642,1.1647,631.0,-658.515152,119.0,-0.476839,0.028786,-0.087163,...,0.000215,0.000313,9.890616,2.506637,7.383978,1.164725,-0.008585,-0.008586,0.000000,0.001030
2015-01-02 12:00:00,1.1648,1.1659,1.1642,1.1659,443.0,-215.515152,562.0,-0.118155,0.100616,0.115124,...,0.001896,0.006723,10.497761,4.104862,6.392898,1.165237,0.103031,0.102978,0.103031,-0.000343
2015-01-02 13:00:00,1.1658,1.1666,1.1655,1.1655,341.0,-556.515152,221.0,-0.257051,0.066757,0.322581,...,0.003941,0.008178,8.706570,5.025204,3.681366,1.165352,-0.034308,-0.034314,0.068687,0.000686
